# Hello World API using Flask

In [2]:
import os
hello_world_script_file = os.path.join(os.pardir, 'src', 'models', 'hello_world_api.py')

In [3]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return 'hello {}'.format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ../src/models/hello_world_api.py


In [4]:
import json
import requests

In [6]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name':'Sebika'})
r = requests.post(url, data)

In [7]:
print(r.text)

hello Sebika


# Machine Learning API using Flask

### Building the API

In [5]:
ml_api_file = os.path.join(os.pardir,'src', 'models', 'machine_learning_api.py')

In [34]:
%%writefile $ml_api_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

# Load Model and Scaler files
model_path = os.path.join(os.pardir, os.pardir, 'models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath, 'rb'))
model = pickle.load(open(model_filepath, 'rb'))

# columns

columns = [u'Age', u'Fare', u'FamilySize', u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', \
          u'Pclass_1', u'Pclass_2', u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', u'Title_Mrs', u'Title_Officer', u'Title_Sir', \
           u'Fare_bin_very_low', u'Fare_bin_low', u'Fare_bin_high', u'Fare_bin_very_high', u'Embarked_C', u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', \
           u'AgeState_Child'
          ]
@app.route('/api',methods = ['POST'])

def make_prediction():
    #read json object and convert to json string
    data = json.dumps(request.get_json(force=True))
    #create pandas dataframe using json string
    df = pd.read_json(data)
    #extract passenger Ids
    passenger_ids = df['PassengerId'].ravel()
    # actual survived values
    actuals = df['Survived'].ravel()
    #extract the required columns and convert to matrix
    X = df[columns].values.astype('float')
    # transform the input
    X_scaled = scaler.transform(X)
    # make predictions 
    predictions = model.predict(X_scaled)
    #create response dataframe
    df_response = pd.DataFrame({'PassengerId':passenger_ids, 'Predicted': predictions, 'Actual':actuals})
    #return json
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=10001, debug=True)
    


Overwriting ../src/models/machine_learning_api.py


### Invoking API using Requests

In [10]:
import os
import pandas as pd

processed_data_path = os.path.join(os.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [11]:
survived_passengers = train_df[train_df['Survived']==1][:5]

In [12]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_bin_very_low,Fare_bin_low,Fare_bin_high,Fare_bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [21]:
import requests

def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make POST request
    r = requests.post(url,data)
    
    return r.json()

In [30]:
make_api_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [32]:
# compare results
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

,PassengerId,Predicted,Actual
0,1,0,0
1,2,1,1
10,108,0,1
100,190,0,0
101,20,1,1


In [36]:
import numpy as np
np.mean(df_result.Actual == df_result.Predicted)

0.8361391694725028